In [2]:
import torch
from torch.distributions import Normal



In [4]:
std_norm_cdf = Normal(0, 1).cdf
std_norm_pdf = lambda x: torch.exp(Normal(0, 1).log_prob(x))

def bs_price(right, K, S, T, sigma, r):
    d_1 = (1 / (sigma * torch.sqrt(T))) * (torch.log(S / K) + (r + (torch.square(sigma) / 2)) * T)
    d_2 = d_1 - sigma * torch.sqrt(T)
    
    if right == "C":
        C = std_norm_cdf(d_1) * S - std_norm_cdf(d_2) * K * torch.exp(-r * T)
        return C
        
    elif right == "P":
        P = std_norm_cdf(-d_2) * K * torch.exp(-r * T) - std_norm_cdf(-d_1) * S
        return P

In [48]:
right = "C"
K = torch.tensor(100.0, requires_grad=True, device="mps")
S = torch.tensor(100.0, requires_grad=True, device="mps")
T = torch.tensor(1.0, requires_grad=True, device="mps")
sigma = torch.tensor(0.40, requires_grad=True, device="mps")
r = torch.tensor(0.05, requires_grad=True, device="mps")

price = bs_price(right, K, S, T, sigma, r)
print(price)

tensor(18.0230, device='mps:0', grad_fn=<SubBackward0>)


In [49]:
d_1 = (1 / (sigma * torch.sqrt(T))) * (torch.log(S / K) + (r + (torch.square(sigma) / 2)) * T)
d_2 = d_1 - sigma * torch.sqrt(T)

delta = std_norm_cdf(d_1)
vega = S * std_norm_pdf(d_1) * torch.sqrt(T)
theta = ((S * std_norm_pdf(d_1) * sigma) / (2 * torch.sqrt(T))) + r * K * torch.exp(-r * T) * std_norm_cdf(d_2)
rho = K * T * torch.exp(-r * T) * std_norm_cdf(d_2)

In [50]:
S = torch.tensor(100.0, requires_grad=True)
price = bs_price(right, K, S, T, sigma, r)

delta = torch.autograd.grad(price, S, create_graph=True)[0]
delta.backward()

print(f"Autograd Gamma: {S.grad}")

# And the direct Black-Scholes calculation
gamma = std_norm_pdf(d_1) / (S * sigma * torch.sqrt(T))
print(f"BS Gamma: {gamma}")

Autograd Gamma: 0.009460493922233582
BS Gamma: 0.00946049578487873


In [51]:
print(f"Delta: {delta}\nVega: {vega}\nTheta: {theta}\nRho: {rho}\nBS Gamma: {gamma}")

Delta: 0.6274095773696899
Vega: 37.84197998046875
Theta: 9.804295539855957
Rho: 44.71799087524414
BS Gamma: 0.00946049578487873


In [52]:
K = torch.tensor(100.0, requires_grad=True)
S = torch.tensor(100.0, requires_grad=True)
T = torch.tensor(1.0, requires_grad=True)
sigma = torch.tensor(0.40, requires_grad=True)
r = torch.tensor(0.05, requires_grad=True)

Z = torch.randn([1000000])
# Brownian Motion
W_T = torch.sqrt(T) * Z
# GBM
prices = S * torch.exp((r - 0.5 * torch.square(sigma)) * T + sigma * W_T)

In [53]:
payoffs = torch.max(prices - K, torch.zeros(1000000))
value = torch.mean(payoffs) * torch.exp(-r * T)
print(value)
value.backward()

tensor(18.0455, grad_fn=<MulBackward0>)


In [54]:
print(f"Delta: {S.grad}\nVega: {sigma.grad}\nTheta: {T.grad}\nRho: {r.grad}")


Delta: 0.6280418038368225
Vega: 37.91283416748047
Theta: 9.820500373840332
Rho: 44.7586784362793


In [57]:
# All the same parameters for the price process
K = torch.tensor(100.0, requires_grad=True)
S = torch.tensor(100.0, requires_grad=True)
T = torch.tensor(1.0, requires_grad=True)
sigma = torch.tensor(0.40, requires_grad=True)
r = torch.tensor(0.05, requires_grad=True)

dt = torch.tensor(1 / 252)
Z = torch.randn([1000000, int(T * 252)])

# Brownian Motion
W_t = torch.cumsum(torch.sqrt(dt) * Z, 1)
# GBM
prices = S * torch.exp((r - 0.5 * torch.square(sigma)) * T + sigma * W_t)

In [58]:
prices

tensor([[100.7104, 100.3935, 100.8341,  ..., 108.6491, 108.5782, 108.2933],
        [100.9128, 101.1900, 101.2585,  ..., 100.8194, 101.0365, 101.2014],
        [101.1213, 100.9249, 100.9966,  ...,  98.5784,  98.3442,  98.0231],
        ...,
        [100.9914, 101.2720, 101.2488,  ..., 104.3196, 104.3497, 104.5994],
        [100.3771, 100.2005, 100.1537,  ...,  99.9048, 100.0811, 100.0502],
        [100.9941, 100.8248, 100.5378,  ..., 108.7409, 108.6943, 108.1600]],
       grad_fn=<MulBackward0>)

In [40]:
Z

tensor([-0.5459,  0.4634, -0.8740,  ..., -0.3667,  1.0500,  0.2378])